In [1]:
%cd ..

/home/mmansour/Music/AUB/Julia/TreeBased_Carla


In [2]:
from copy import deepcopy
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

import warnings

import pandas as pd

warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.filterwarnings("ignore")
from carla.data.api import data
import numpy as np
import torch
torch.cuda.is_available = lambda : False
import yaml
from seed_env import seed_my_session
from typing import Dict, List
from cent.data_specific import DataModels
from carla import Benchmark
import pandas as pd
from carla.recourse_methods import (
    CCHVAE,
    CEM,
    CRUD,
    FOCUS,
    CausalRecourse,
    Clue,
    Dice,
    Face,
    FeatureTweak,
    GrowingSpheres,
    Revise,
    Wachter,
)
from carla.recourse_methods.catalog.causal_recourse import constraints, samplers
import carla.evaluation.catalog as evaluation_catalog
from cent.method import CEnt
from vae_benchmark import VAEBenchmark
from tensorflow import Graph, Session
from carla.models.catalog import MLModelCatalog

Using TensorFlow backend.


[INFO] Using Python-MIP package version 1.12.0 [model.py <module>]
[WARNING] From /tmp/ipykernel_52171/3118733197.py:40: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.
 [deprecation_wrapper.py __getattr__]


In [3]:
data_name = 'adult'
FACTUAL_NUMBER = 20
OUT_DIR_DATA = 'outputs/tmp/'
data_models = DataModels(data_name = data_name,
                             factuals_length = FACTUAL_NUMBER,
                             out_dir = OUT_DIR_DATA)

Loading models... --- logs will be saved to outputs/tmp/models_logs.txt
[WARNING] From /home/mmansour/miniconda3/envs/xai/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:97: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor [deprecation.py new_func]
[WARNING] From /home/mmansour/miniconda3/envs/xai/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor [deprecation.py new_func]


In [4]:
layers = [25, 16]
latent_dim = 8
mlmodel = data_models.models_zoo['ann']['tensorflow']

In [5]:
min_entries_per_label = int(data_models.trainData.df.shape[0]*0.01)
MIN_ENTRIES_PER_LABEL_THRESH = 500
if min_entries_per_label<MIN_ENTRIES_PER_LABEL_THRESH:
    print('min_entries_per_label is too small {}, setting it to {} '.format(min_entries_per_label,MIN_ENTRIES_PER_LABEL_THRESH))
#TODO: @MM Return this to 1% of data
min_entries_per_label = MIN_ENTRIES_PER_LABEL_THRESH
hpr = {"data_name": "data_name","n_search_samples": 300,
        "p_norm": 1,"step": 0.1,"max_iter": 10,"clamp": True,
        "treeWarmUp": 5,
        "binary_cat_features": True,
        "myvae_params": {
            'input_dim': len(mlmodel.feature_input_order),
            'kld_weight': 0.00025,
            'layers': layers,
            'latent_dim': latent_dim,
            'hidden_activation': 'relu',
            'dropout': 0.2,
            'batch_norm': True,
            'batch_size': 32,
            'epochs': 1,
            'learning_rate': 0.001,
            'weight_decay': 0.000001,
            'cuda': False,
            'verbose': True,
            'train': True,
            'save_dir': './vae_model/',
        },
        "tree_params": {
            "min_entries_per_label": min_entries_per_label,
            "grid_search_jobs": -1,
            "min_weight_gini": 100,
            "max_search" : 50,
            "grid_search": {"cv": 1,"splitter": ["best"],"criterion": ["gini"],"max_depth": [3,4,5,6,7],
                            "min_samples_split": [1.0,2,3],"min_samples_leaf": [1,2,3],
                            "max_features": ['sqrt',1.0, 'log2',0.8],
                            }
        }
    }
cent_method =  CEnt(deepcopy(data_models.trainData), mlmodel, hpr, data_catalog= data_models.new_catalog_n)

min_entries_per_label is too small 487, setting it to 500 
{'input_dim': 13, 'kld_weight': 0.00025, 'layers': [25, 16], 'latent_dim': 8, 'hidden_activation': 'relu', 'dropout': 0.2, 'batch_norm': True, 'batch_size': 32, 'epochs': 1, 'learning_rate': 0.001, 'weight_decay': 1e-06, 'cuda': False, 'verbose': True, 'train': True, 'save_dir': './vae_model/'}
./vae_model/adult
Epoch: 0, ELBO Loss: 43.046390533447266, Test MSELoss: 23.748332977294922
Epoch: 0, Best ELBO Loss: 43.046390533447266
Get Encodings...
Initializing the NNDescent...
DT Warming Up on 5 fits...
{'criterion': 'gini', 'max_depth': 5, 'max_features': 1.0, 'min_samples_leaf': 3, 'min_samples_split': 2, 'splitter': 'best'}


In [9]:
fact = data_models.factuals['ann'].sample(1)
cent_method.get_counterfactuals(fact)

,age,fnlwgt,education-num,capital-gain,capital-loss,...,occupation_Other,relationship_Non-Husband,race_White,sex_Male,native-country_US
15671,0.273973,0.061062,0.766881,0.047293,0.0,...,1.0,1.0,0.0,0.0,1.0


In [18]:
print("To flip the class from {} to {}, we applied the following:".format(fact[mlmodel.data.target].values[0],fact[mlmodel.data.target].values[0]*-1 +1))
print(cent_method.found_node.conditions)


To flip the class from 0 to 1, we applied the following:
[education-num <= 0.7666666805744171, capital-gain > 0.04831548407673836, education-num > 0.5666666924953461]
